In [37]:

import torch

import torch.nn as nn


input_size = 3  # размерность входного вектора
hidden_size = 20  # размерность скрытого состояния
num_layers = 2   # количество рекуррентных слоёв
rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
batch_size = 3
seq_length = 5
x = torch.randn(batch_size, seq_length, input_size)
h0 = torch.zeros(num_layers, batch_size, hidden_size)
output, hn = rnn(x, h0)
print('Tensor output:', output.shape)
print(output)
print('Last hn:', hn.shape)
print(hn)



Tensor output: torch.Size([3, 5, 20])
tensor([[[-0.1136, -0.0677,  0.0240, -0.2410, -0.2703, -0.2916, -0.1610,
          -0.0403, -0.2039,  0.3268,  0.0936, -0.1283,  0.4434,  0.1891,
           0.1537,  0.1830, -0.0283, -0.0970,  0.1833, -0.2228],
         [-0.2771,  0.1065, -0.0923,  0.1841, -0.4071, -0.0253, -0.4378,
          -0.0659,  0.0220,  0.0691,  0.2979, -0.2294,  0.4487,  0.0480,
           0.1620,  0.0849, -0.0809, -0.3667,  0.2824, -0.4627],
         [-0.1167,  0.3501,  0.1699, -0.0840, -0.3866, -0.1472, -0.5147,
          -0.3408,  0.0582,  0.0045,  0.0642, -0.3626,  0.5542,  0.1922,
           0.4794,  0.1559,  0.0337, -0.3902,  0.1412, -0.4100],
         [-0.0065,  0.2015, -0.2479, -0.3228, -0.0819, -0.3216, -0.3709,
          -0.4735,  0.1980,  0.1359,  0.3047, -0.2625,  0.5946,  0.3553,
           0.3621,  0.2191,  0.0708, -0.4177,  0.3053, -0.1882],
         [-0.1203,  0.0264,  0.0337,  0.1757, -0.5121, -0.2706, -0.3242,
          -0.2235, -0.0763, -0.0206,  0.1703,

In [41]:
vocab_size = 2000
embedding_dim = 3

emb_layer = nn.Embedding(vocab_size, embedding_dim)
rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
emb_layer_res = emb_layer(torch.tensor([[2, 3, 1523, 7], [123, 46, 56, 2]]))
rnn_layer_res = rnn(emb_layer_res)


In [43]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

lengths = torch.tensor([5, 3, 2])
packed_x = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
packed_output, hn = rnn(packed_x, None)
output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)

In [3]:
import pandas as pd

raw = pd.read_csv('/home/user/Загрузки/yelp_reviews.csv')
texts = raw['text']
labels = raw['label']
print("Size dataset's:")
print(len(texts))
print('First feedback from dataset:')
print(texts[0])
print("And it's rating:")
print(labels[0])





Size dataset's:
6500
First feedback from dataset:
Worst sandwich on Earth.\nI'd rather eat a dead whore.\nPlease...never come here.
And it's rating:
0


In [6]:
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence


# загрузка датасета
raw = pd.read_csv('yelp_reviews.csv')


texts = raw['text']
labels = raw['label']


# разделение выборки на трейн и тест
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)


# создание токенизатора с помощью класса AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = # создайте токенизатор


# токенизируем тексты
train_texts_tokenized = tokenizer(train_texts, truncation=True)['input_ids']
val_texts_tokenized = tokenizer(val_texts, truncation=True)['input_ids']


# создаём класс кастомного, наследуясь от класса Dataset из PyTorch

class YelpDataset(Dataset):
    # в конструкторе просто сохраняем тексты и классы
    def __init__(self, texts, labels, max_len=256):
        self.texts = # сохраните тексты
        self.labels = # сохраните классы
        self.max_len = # сохрание max_len


    # возвращаем размер датасета (кол-во текстов)
    def __len__(self):
        return # верните размер датасета
        
    def __getitem__(self, idx):
        # возвращаем текст и его класс
        # для текста ограничиваем длину
        # не делаем никаких доп. преобразований как padding и masking
        return {
            'text': # верните текст под индексом idx в виде тензора, ограничьте его длиной self.max_len
            'label': # верните класс под индексом idx в виде тензора
        }


# кастомная функция collate_fn для формирования батчей
def collate_fn(batch):
    texts = # получите список текстов в батче
    labels = # получите список классов в батче
    lengths = # посчитайте список длин текстов в батче 
    padded_texts = # реализуйте пэддинг для текстов


    return {
        'input_ids': padded_texts, 
        'lengths': lengths, 
        'labels': labels
    }


# создаём датасеты
train_dataset = YelpDataset(texts=train_texts_tokenized, labels=train_labels)
val_dataset = YelpDataset(texts=val_texts_tokenized, labels=val_labels)


batch_size = 64


# создаём даталоадеры
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


print(f'Количество батчей в train_dataloader: {len(train_dataloader)}')
print(f'Количество батчей в val_dataloader: {len(val_dataloader)}')


print('Размерности батчей:')
for batch in train_dataloader:
    print('input_ids:', batch['input_ids'].shape)
    print('lengths:', batch['lengths'].shape)
    print('labels:', batch['labels'].shape)
    break

In [7]:
import sys
print("Текущий интерпретатор Python:", sys.executable)

Текущий интерпретатор Python: /home/user/Projects/sprint1/.venv/bin/python
